<a href="https://colab.research.google.com/github/solobala/numpy36/blob/main/Balakina_numpy36_DZ2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание «Библиотека Pandas»

In [ ]:
import pandas as pd

# Задание 1
Скачайте с сайта grouplens.org...movielens/ датасет любого размера. Определите, какому фильму было выставлено больше всего оценок 5.0.

In [ ]:
!wget -O "ml-latest-small.zip" "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"   # Качаем архив выбранного датасета

--2022-08-17 07:36:27--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.57MB/s    in 0.2s    

2022-08-17 07:36:28 (4.57 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [ ]:
from zipfile import ZipFile  # Подключаем библиотеку ZipFile из пакета zipfile

In [ ]:
with ZipFile('ml-latest-small.zip', 'r') as zip_file:  # Распаковываем архив в сессионное хранилище в папку ml-latest-small. Нам понадобятся файлы ratings.csv и movies.csv
     #zip_file.extractall('/content/')
    print(zip_file.infolist())
    zip_file.extract('ml-latest-small/ratings.csv')
    zip_file.extract('ml-latest-small/movies.csv')

[<ZipInfo filename='ml-latest-small/' filemode='drwxr-xr-x' external_attr=0x10>, <ZipInfo filename='ml-latest-small/links.csv' compress_type=deflate filemode='-rw-r--r--' file_size=197979 compress_size=87110>, <ZipInfo filename='ml-latest-small/tags.csv' compress_type=deflate filemode='-rw-r--r--' file_size=118660 compress_size=36263>, <ZipInfo filename='ml-latest-small/ratings.csv' compress_type=deflate filemode='-rw-r--r--' file_size=2483723 compress_size=684231>, <ZipInfo filename='ml-latest-small/README.txt' compress_type=deflate filemode='-rw-r--r--' file_size=8342 compress_size=3285>, <ZipInfo filename='ml-latest-small/movies.csv' compress_type=deflate filemode='-rw-r--r--' file_size=494431 compress_size=166235>]


In [ ]:
data_ratings = pd.read_csv('/content/ml-latest-small/ratings.csv') # Читаем файл raiting.csv

In [ ]:
data_ratings_filtered =data_ratings[data_ratings['rating']==5.0]  # отфильтровываем фильмы с оценками 5.0

In [ ]:
# Ищем идентификатор фильма с наибольшим к-вом пятерок. 
# для этого группируем фильмы по movieId, 
# считаем кол-во пятерок для каждого идентификаторы, 
# сортируем по убыванию и берем только 1-ю строку

my_filter = data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1).index.values[0]
type(data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1))
#my_filter

pandas.core.frame.DataFrame

In [ ]:
my_filter = data_ratings_filtered.movieId.value_counts().head(1).index.values[0]  # другой способ. Но тогда, Mergeииспользовать нельзяЮ т.к получается не датафрейм, а серия
my_filter
type(data_ratings_filtered.movieId.value_counts().head(1)) 


pandas.core.series.Series

In [ ]:
data = pd.read_csv('/content/ml-latest-small/movies.csv') # теперь читаем файл movies.csv с названиями фильмов

In [ ]:
print(f"Фильм с наибольшим количеством оценок 5.0: {data[data['movieId'] == my_filter]['title'].values[0]}")

Фильм с наибольшим количеством оценок 5.0: Shawshank Redemption, The (1994)


In [ ]:
# другой способ - решение через merge

joined = data_ratings_filtered[['movieId', 'rating']].groupby('movieId').count().sort_values(by = 'rating', ascending = False).head(1).merge(data, on ='movieId', how = 'left')


In [ ]:
print(f"Фильм с наибольшим количеством оценок 5.0: {joined['title'].values[0]}")

Фильм с наибольшим количеством оценок 5.0: Shawshank Redemption, The (1994)


# Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [ ]:
def baltic(country):
    """Объединение стран Прибалтики
      не использовать, если используем
      my_filter
    """
    if country in ['lithuania', 'Latvia', 'Estonia']:
      return 'Прибалтика'
    return 'Other'

In [ ]:
def my_categories(category):
    """ Отбор категорий потребления согласно критерию 
    не использовать, если используем
      my_filter
    """
    if category in [4, 12, 21]:
      return "selected"
    return "Other"

In [ ]:
def my_filter(row): #быстрее ищет по множеству,Ж а не по списку, но здесь маленькая длина списка, нет смысла
    """Отбор сразу по всем критериям - вхождению в Прибалтику, 
    нужной категории, году и неотрицатиельному количеству"""
    return row['country'] in ['lithuania', 'Latvia', 'Estonia'] \
      and  row['category'] in [4, 12, 21] \
      and row['year'] in range(2005,2011) \
      and row['quantity'] >= 0
    

In [ ]:
data = pd.read_csv('/content/power.csv')
data.head()

,country,year,quantity,category
0,Austria,1996,5.0,1.0
1,Austria,1995,17.0,1.0
2,Belgium,2014,0.0,1.0
3,Belgium,2013,0.0,1.0
4,Belgium,2012,35.0,1.0


In [ ]:
data['my_categories'] = data.category.apply(my_categories) #не исп., если исп. my_filter


In [ ]:
data['baltic'] = data.country.apply(baltic) #не исп., если исп. my_filter

In [ ]:
filtered_data = data [(data['baltic'] == 'Прибалтика')  #не исп., если исп. my_filter
& (data['my_categories'] == 'selected') 
& (data['year'] > 2004)  
& (data['year'] < 2011) 
& (data['quantity']>=0)]



In [ ]:
data['my_filter'] = data.apply(my_filter, axis = 1)

In [ ]:
# то же самое без def, но с использованием lambda - функции
data['my_filter'] = data.apply(lambda x: x['country'] in ['lithuania', 'Latvia', 'Estonia'] \
      and x['category'] in [4, 12, 21] \
      and x['year'] in range(2005,2011) \
      and x['quantity'] >= 0 ,axis = 1)

In [ ]:
filtered_data = data [(data['my_filter'] == True)] 

In [ ]:
print(f" суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: {sum(filtered_data['quantity'])}")

 суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года: 118284.0


# Задание 3
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
www.finanz.ru...om-vremeni

In [ ]:
page_url = 'https://www.finanz.ru/aktsii/polymetal'

In [ ]:
df = pd.read_html(page_url, attrs ={'class':'header_user_pricebox'}, encoding = 'utf-8')

In [ ]:
df

[     0       1     2        3       4      5        6       7      8   \
 0  ММВБ   2 137  2,3%   Nasdaq  12 494  -1,2%  Биткойн  22 883  -1,3%   
 1   РТС   1 116  3,0%   S&P500   4 122  -0,4%    Нефть     961  -0,5%   
 2   Dow  32 833  0,1%  FTSE100   7 488   0,1%   Золото   1 794   0,0%   
 
         9       10    11  
 0  USD/RUB  604750  0,0%  
 1  EUR/RUB  617903  0,0%  
 2  EUR/USD   10218  0,0%  ]